<a href="https://colab.research.google.com/github/sahandfrd94/Image-classification-by-keras/blob/main/Image_classification_by_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import dataset

In [1]:
!wget https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip


--2024-02-04 22:18:55--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.44.78.15, 2600:1407:7400:489::317f, 2600:1407:7400:481::317f
Connecting to download.microsoft.com (download.microsoft.com)|23.44.78.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824887076 (787M) [application/octet-stream]
Saving to: ‘kagglecatsanddogs_5340.zip’

kagglecatsanddogs_5 100%[===================>] 786.67M   118MB/s    in 8.7s    

2024-02-04 22:19:04 (90.0 MB/s) - ‘kagglecatsanddogs_5340.zip’ saved [824887076/824887076]



In [2]:
! unzip -qx /content/kagglecatsanddogs_5340.zip

In [3]:
#removing empty files
from tqdm.notebook import tqdm
from glob import glob as g
import os
count = 0
for name in tqdm(g('/content/PetImages/*/*')):

  if not os.path.getsize(name):
    count += 1
    os.remove(name)
print(count, ' empty files removed')

  0%|          | 0/25002 [00:00<?, ?it/s]

2  empty files removed


# Create train & test data

In [4]:
cat_train_loc = '/content/CAT & DOG/train/cat'
cat_val_loc = '/content/CAT & DOG/validation/cat'
dog_train_loc = '/content/CAT & DOG/train/dog'
dog_val_loc = '/content/CAT & DOG/validation/dog'

data_locations = [cat_train_loc,cat_val_loc,dog_train_loc,dog_val_loc]
for loc in data_locations:
  os.makedirs(loc)


In [5]:
# take the addres of all image
cats = sorted(g('/content/PetImages/Cat/*'))
dogs = sorted(g('/content/PetImages/Dog/*'))

In [6]:
from sklearn.model_selection import train_test_split
cat_train , cat_test = train_test_split(cats , test_size = 0.2 , random_state = 101 , shuffle = True)
dog_train , dog_test = train_test_split(dogs , test_size = 0.2 , random_state = 101 , shuffle = True)

In [7]:
# import images in locations
from shutil import copy

In [8]:
for img in tqdm(cat_train):
  copy(img ,cat_train_loc )
for img in tqdm(dog_train):
  copy(img , dog_train_loc)
for img in tqdm(cat_test):
  copy(img , cat_val_loc)
for img in tqdm(dog_test):
  copy(img , dog_val_loc)

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/2500 [00:00<?, ?it/s]

In [9]:
print(len(os.listdir(cat_train_loc)))
print(len(os.listdir(cat_val_loc)))
print(len(os.listdir(dog_train_loc)))
print(len(os.listdir(dog_val_loc)))

10000
2500
10000
2500


In [10]:
from  tensorflow.keras.preprocessing.image import ImageDataGenerator
train_scaler = ImageDataGenerator(rescale = 1/255)
train_gen = train_scaler.flow_from_directory(
    '/content/CAT & DOG/train',
    batch_size = 64,
    class_mode= 'binary',
    target_size = (160,160)
)

test_scaler  = ImageDataGenerator(rescale = 1/255)
val_gen = test_scaler.flow_from_directory(
    '/content/CAT & DOG/validation',
    batch_size = 64,
    class_mode = 'binary',
    target_size  = (160 , 160)

)

Found 19998 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


# Creat CNN model

In [11]:
from keras.models import Sequential
from keras.layers import Dense , Dropout , MaxPool2D , Flatten , Conv2D

In [12]:
out_put = next(train_gen)

In [13]:
out_put[0].shape

(64, 160, 160, 3)

In [14]:
print(val_gen.class_indices)

{'cat': 0, 'dog': 1}


In [15]:
model = Sequential()
model.add(Conv2D(64, (3,3), activation = 'relu', input_shape = (160,160,3), padding = 'same', kernel_initializer = 'he_uniform'))
model.add(Conv2D(64, (4,4), activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform'))

model.add(Dropout(0.2))
model.add(MaxPool2D(2,2))

model.add(Conv2D(128, (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform'))
model.add(Conv2D(128, (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform'))


model.add(Dropout(0.2))
model.add(MaxPool2D(2,2))

model.add(Conv2D(256, (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform'))
model.add(Conv2D(256, (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform'))

model.add(Dropout(0.3))
model.add(MaxPool2D(2,2))

model.add(Flatten())

model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation = 'sigmoid'))

In [16]:
model.compile(loss = 'binary_crossentropy' , metrics=['accuracy'] , optimizer = 'adam')

In [17]:
history = model.fit(
    train_gen,
    epochs = 30,
    batch_size  = 128,
    verbose = 1,
    validation_data = val_gen
)

Epoch 1/30
313/313 [==============================] - ETA: 0s - loss: 1.2930 - accuracy: 0.5363

/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:858: UserWarning: Truncated File Read
  warnings.warn(str(msg))


313/313 [==============================] - 174s 482ms/step - loss: 1.2930 - accuracy: 0.5363 - val_loss: 0.6926 - val_accuracy: 0.5108
Epoch 2/30
313/313 [==============================] - 138s 440ms/step - loss: 0.7215 - accuracy: 0.5211 - val_loss: 0.6890 - val_accuracy: 0.5070
Epoch 3/30
313/313 [==============================] - 139s 445ms/step - loss: 0.6640 - accuracy: 0.6050 - val_loss: 0.6396 - val_accuracy: 0.6450
Epoch 4/30
313/313 [==============================] - 139s 444ms/step - loss: 0.6099 - accuracy: 0.6734 - val_loss: 0.6280 - val_accuracy: 0.6632
Epoch 5/30
313/313 [==============================] - 139s 445ms/step - loss: 0.5523 - accuracy: 0.7202 - val_loss: 0.5376 - val_accuracy: 0.7474
Epoch 6/30
313/313 [==============================] - 140s 447ms/step - loss: 0.4884 - accuracy: 0.7686 - val_loss: 0.5221 - val_accuracy: 0.7678
Epoch 7/30
313/313 [==============================] - 141s 449ms/step - loss: 0.4273 - accuracy: 0.8054 - val_loss: 0.4826 - val_accura